# Retrieval Augmented Generation (RAG) Agent

We built a simple RAG in another notebook. This notebook build RAG using a react agent.

Compared to the simple RAG, here retrieval is decided by the LLM during generation. This allows better queries (as the LLM can simply and rephrase the inputs into better queries), and possibly multiple retrievals (or no retrieval at all if the LLM thinks that it is not necessary). One drawback is that we need an LLM that supports tool use natively.

We will use [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss/) as the vector database (vectorstore).

If you are going for a more complicated multi-agent retriever, try [this LangGraph tutorial](https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag/#graph).

In [1]:
# !pip install faiss-cpu langchain_community langchain_openai pandas

In [2]:
import os
from uuid import uuid4

import faiss
import pandas as pd
from dotenv import load_dotenv
from langchain_community.docstore import InMemoryDocstore
from langchain_community.embeddings import LlamafileEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

In [3]:
load_dotenv()

True

## Load dataset and generate documents

We use the [Complete Pokédex dataset](https://www.kaggle.com/datasets/cristobalmitchell/pokedex/data), which is a dataset about Pokemon
but unfortunately not that complete.

I've included a variant of it in the `data` folder, which I save the original file using comma
separation and UTF-8 since the original file is hard to open directly using pandas.

In [4]:
df = pd.read_csv("../data/pokemon.csv")

In [5]:
df

,national_number,gen,english_name,japanese_name,primary_type,secondary_type,classification,percent_male,percent_female,height_m,...,evochain_1,evochain_2,evochain_3,evochain_4,evochain_5,evochain_6,gigantamax,mega_evolution,mega_evolution_alt,description
0,1,I,Bulbasaur,Fushigidane,grass,poison,Seed Pokémon,88.14,11.86,0.7,...,Level,Ivysaur,Level,Venusaur,NaN,NaN,NaN,NaN,NaN,There is a plant seed on its back right from t...
1,2,I,Ivysaur,Fushigisou,grass,poison,Seed Pokémon,88.14,11.86,1.0,...,Level,Ivysaur,Level,Venusaur,NaN,NaN,NaN,NaN,NaN,"When the bulb on its back grows large, it appe..."
2,3,I,Venusaur,Fushigibana,grass,poison,Seed Pokémon,88.14,11.86,2.0,...,Level,Ivysaur,Level,Venusaur,NaN,NaN,Gigantamax Venusaur,Mega Venusaur,NaN,Its plant blooms when it is absorbing solar en...
3,4,I,Charmander,Hitokage,fire,NaN,Lizard Pokémon,88.14,11.86,0.6,...,Level,Charmeleon,Level,Charizard,NaN,NaN,NaN,NaN,NaN,It has a preference for hot things. When it ra...
4,5,I,Charmeleon,Lizardo,fire,NaN,Flame Pokémon,88.14,11.86,1.1,...,Level,Charmeleon,Level,Charizard,NaN,NaN,NaN,NaN,NaN,"It has a barbaric nature. In battle, it whips ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,894,VIII,Regieleki,Regieleki,electric,NaN,Electron Pokémon,NaN,NaN,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,This Pokémon is a cluster of electrical energy...
894,895,VIII,Regidrago,Regidrago,dragon,NaN,Dragon Orb Pokémon,NaN,NaN,2.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An academic theory proposes that Regidrago’s a...
895,896,VIII,Glastrier,Blizapos,ice,NaN,Wild Horse Pokémon,NaN,NaN,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glastrier emits intense cold from its hooves. ...
896,897,VIII,Spectrier,Rayspos,ghost,NaN,Swift Horse Pokémon,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,It probes its surroundings with all its senses...


In [6]:
def generate_document(row):
    doc = f"""Number: {row["national_number"]:03}
Generation: {row["gen"]}
Name: {row["english_name"]}
Type: {row["primary_type"]}{" / " + row["secondary_type"] if pd.notnull(row["secondary_type"]) else ""}
Species: {row["classification"]}
Abilities: {", ".join([x for x in [row["abilities_0"], row["abilities_1"], row["abilities_2"], row["abilities_hidden"]] if pd.notnull(x)])}
Evolutions: {" -> ".join([x for x in [row["evochain_0"], row["evochain_2"], row["evochain_4"], row["evochain_6"]] if pd.notnull(x)])}
Description: {row["description"]}"""

    return Document(page_content=doc, metadata={})

In [7]:
df["document"] = df.apply(generate_document, axis=1)

In [8]:
print(df.loc[0, "document"].page_content)

Number: 001
Generation: I
Name: Bulbasaur
Type: grass / poison
Species: Seed Pokémon
Abilities: Overgrow, Chlorophyll
Evolutions: Bulbasaur -> Ivysaur -> Venusaur
Description: There is a plant seed on its back right from the day this Pokémon is born. The seed slowly grows larger.


To shorten run time, we will only use pokemon from gen 1. If you have more computation power, or have access to a faster API for embeddings, you can use the full dataset.

In [9]:
df = df[df["gen"] == "I"]

## Define embedding and model

We are still using the `mxbai-embed-large-v1` Llamafile for embeddings, but this time we use `gpt-4o` for the model. We need a stronger model here due to tool use.

In [10]:
embeddings = LlamafileEmbeddings()

llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_API_ENDPOINT"),
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4o",
    temperature=0,
)

## Define vectorstore

In [11]:
# embed_dim = len(embeddings.embed_query("Hello World"))
# index = faiss.IndexFlatL2(embed_dim)

# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

## Load documents into vectorstore

Transform to langchain's Document format, which is suitable for the vector store to consume. Note that our documents are very short, so no chunking is necessary.

In [12]:
# documents = list(df["document"])
# uuids = [str(uuid4()) for _ in range(len(documents))]
# for document, uid in zip(tqdm(documents), uuids):
#     vector_store.add_documents(documents=[document], ids=[uid])

In [13]:
# save and local vectorstore

# vector_store.save_local("../pokemon_faiss_index")

vector_store = FAISS.load_local(
    "../pokemon_faiss_index", embeddings, allow_dangerous_deserialization=True
)

Test the vectorstore a bit.

In [14]:
results = vector_store.similarity_search_with_score(
    "Which pokémon likes sleeping?", k=3, filter={}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content}")

* [SIM=0.629833] Number: 061
Generation: I
Name: Poliwhirl
Type: water
Species: Tadpole Pokémon
Abilities: Water Absorb, Damp, Swift Swim
Evolutions: Poliwag -> Poliwhirl -> Poliwrath -> Politoed
Description: Staring at the swirl on its belly causes drowsiness. This trait of Poliwhirl’s has been used in place of lullabies to get children to go to sleep.
* [SIM=0.669940] Number: 097
Generation: I
Name: Hypno
Type: psychic
Species: Hypnosis Pokémon
Abilities: Insomnia, Forewarn, Inner Focus
Evolutions: Drowzee -> Hypno
Description: Avoid eye contact if you come across one. It will try to put you to sleep by using its pendulum.
* [SIM=0.689354] Number: 143
Generation: I
Name: Snorlax
Type: normal
Species: Sleeping Pokémon
Abilities: Immunity, Thick Fat, Gluttony
Evolutions: Munchlax -> Snorlax
Description: It is not satisfied unless it eats over 880 pounds of food every day. When it is done eating, it goes promptly to sleep.


Make the vectorstore a LangChain retriever for chaining.

In [15]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 20, "lambda_mult": 0.5},
)

retriever_tool = create_retriever_tool(
    retriever,
    name="search_pokemon",
    description="A pokemon search engine. Input is any text, output is the basic information of pokemons.",
)

In [16]:
retr_res = retriever.invoke("Which pokémon is the cutest?")
for doc in retr_res:
    print(doc.page_content)
    print("----")

Number: 039
Generation: I
Name: Jigglypuff
Type: normal / fairy
Species: Balloon Pokémon
Abilities: Cute Charm, Competitive, Friend Guard
Evolutions: Igglybuff -> Jigglypuff -> Wigglytuff
Description: Jigglypuff has top-notch lung capacity, even by comparison to other Pokémon. It won’t stop singing its lullabies until its foes fall asleep.
----
Number: 127
Generation: I
Name: Pinsir
Type: bug
Species: Stagbeetle Pokémon
Abilities: Hyper Cutter, Mold Breaker, Moxie
Evolutions: Pinsir
Description: These Pokémon judge one another based on pincers. Thicker, more impressive pincers make for more popularity with the opposite gender.
----
Number: 025
Generation: I
Name: Pikachu
Type: electric
Species: Mouse Pokémon
Abilities: Static, Lightning Rod
Evolutions: Pichu -> Pikachu -> Raichu
Description: Pikachu that can generate powerful electricity have cheek sacs that are extra soft and super stretchy.
----


## Define agent

In [17]:
# prompt = PromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
# If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
# Question: {question} 
# Context: {context} 
# Answer:
# """)

In [18]:
rag_agent = create_react_agent(llm, [retriever_tool])

## Invoke

In [19]:
events = []
for s in rag_agent.stream(
    {
        "messages": [
            (
                "user",
                "Which pokémon likes sleeping the most?",
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 10},
):
    events.append(s)
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8kN8Euv63WTT8y9P15XG3VPU', 'function': {'arguments': '{"query":"pokémon that likes sleeping"}', 'name': 'search_pokemon'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 74, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_82ce25c0d4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-fd4dfbdb-67c3-424e-bcf9-f879f5f08f58-0', tool_calls=[{'name': 'search_pokemon', 'args': {'query': 'pokémon that likes sleeping'}, 'id': 'call_8kN8Euv63WTT8y9P15XG3VPU', 'type': 'tool_call'}], usage_metadata={'input_tokens': 74, 'output_tokens': 19, 'total_tokens': 93, 'input_token_details': {'audio': 0, 

In [20]:
for s in events:
    sender = list(s.keys())[0]
    for message in s[sender]["messages"]:
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_pokemon (call_8kN8Euv63WTT8y9P15XG3VPU)
 Call ID: call_8kN8Euv63WTT8y9P15XG3VPU
  Args:
    query: pokémon that likes sleeping
================================= Tool Message =================================
Name: search_pokemon

Number: 061
Generation: I
Name: Poliwhirl
Type: water
Species: Tadpole Pokémon
Abilities: Water Absorb, Damp, Swift Swim
Evolutions: Poliwag -> Poliwhirl -> Poliwrath -> Politoed
Description: Staring at the swirl on its belly causes drowsiness. This trait of Poliwhirl’s has been used in place of lullabies to get children to go to sleep.

Number: 143
Generation: I
Name: Snorlax
Type: normal
Species: Sleeping Pokémon
Abilities: Immunity, Thick Fat, Gluttony
Evolutions: Munchlax -> Snorlax
Description: It is not satisfied unless it eats over 880 pounds of food every day. When it is done eating, it goes promptly to sleep.

Number: 063
Generation: I
Name: Abra
Type

Note how the LLM simplifies the query instead of directly put in the question to the vectorstore. Snorlax is the right answer.

In [21]:
events2 = []
for s in rag_agent.stream(
    {
        "messages": [
            (
                "user",
                "Which pokémon is the cutest?",
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 10},
):
    events2.append(s)
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='The "cutest" Pokémon is subjective and depends on personal preferences! Some popular choices for cute Pokémon include:\n\n- **Pikachu**: The iconic mascot of Pokémon, known for its adorable cheeks and playful personality.\n- **Eevee**: Loved for its fluffy tail and big, expressive eyes.\n- **Jigglypuff**: Known for its round, pink body and sweet singing voice.\n- **Togepi**: A baby Pokémon with a cute eggshell design.\n- **Piplup**: A penguin-like Pokémon with a charming demeanor.\n- **Sylveon**: A Fairy-type evolution of Eevee with a pastel, ribbon-like appearance.\n\nIf you\'d like, I can look up specific Pokémon to see their details and help you decide! Let me know.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 163, 'prompt_tokens': 73, 'total_tokens': 236, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}

In [22]:
for s in events2:
    sender = list(s.keys())[0]
    for message in s[sender]["messages"]:
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

================================== Ai Message ==================================

The "cutest" Pokémon is subjective and depends on personal preferences! Some popular choices for cute Pokémon include:

- **Pikachu**: The iconic mascot of Pokémon, known for its adorable cheeks and playful personality.
- **Eevee**: Loved for its fluffy tail and big, expressive eyes.
- **Jigglypuff**: Known for its round, pink body and sweet singing voice.
- **Togepi**: A baby Pokémon with a cute eggshell design.
- **Piplup**: A penguin-like Pokémon with a charming demeanor.
- **Sylveon**: A Fairy-type evolution of Eevee with a pastel, ribbon-like appearance.

If you'd like, I can look up specific Pokémon to see their details and help you decide! Let me know.


The agent is confident in this question and does not use tool at all!

In [23]:
events3 = []
for s in rag_agent.stream(
    {
        "messages": [
            (
                "user",
                "Which pokémon is the cutest? Make sure you consult the search_pokemon tool and does not make any guess yourself.",
            )
        ],
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 10},
):
    events3.append(s)
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LSkYFuT8OFsCKgHT73HzQIfL', 'function': {'arguments': '{"query":"cutest Pokémon"}', 'name': 'search_pokemon'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 90, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_82ce25c0d4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87e2081c-ee33-4f1b-8cdf-e19235b30c96-0', tool_calls=[{'name': 'search_pokemon', 'args': {'query': 'cutest Pokémon'}, 'id': 'call_LSkYFuT8OFsCKgHT73HzQIfL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 17, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'outpu

In [24]:
for s in events3:
    sender = list(s.keys())[0]
    for message in s[sender]["messages"]:
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  search_pokemon (call_LSkYFuT8OFsCKgHT73HzQIfL)
 Call ID: call_LSkYFuT8OFsCKgHT73HzQIfL
  Args:
    query: cutest Pokémon
================================= Tool Message =================================
Name: search_pokemon

Number: 039
Generation: I
Name: Jigglypuff
Type: normal / fairy
Species: Balloon Pokémon
Abilities: Cute Charm, Competitive, Friend Guard
Evolutions: Igglybuff -> Jigglypuff -> Wigglytuff
Description: Jigglypuff has top-notch lung capacity, even by comparison to other Pokémon. It won’t stop singing its lullabies until its foes fall asleep.

Number: 127
Generation: I
Name: Pinsir
Type: bug
Species: Stagbeetle Pokémon
Abilities: Hyper Cutter, Mold Breaker, Moxie
Evolutions: Pinsir
Description: These Pokémon judge one another based on pincers. Thicker, more impressive pincers make for more popularity with the opposite gender.

Number: 025
Generation: I
Name: Pikachu
Type: ele